In [ ]:
from losses import UMAP_precomputed_loss, TSNE_precomputed_loss
from autoneb import autoneb

In [ ]:
x_data = X_train_fashionmnist # load your preferred dataset here, x values
y_data = y_train_fashionmnist # load your preferred dataset here, labels

In [ ]:
# several instances of the UMAP_precomputed_loss class with different parameters can exist simultaneously
fashion_UMAP_loss = UMAP_precomputed_loss(x_data, y_data)

# several instances of the TSNE_precomputed_loss class with different parameters can exist simultaneously
fashion_TSNE_loss = TSNE_precomputed_loss(x_data, y_data)

# run autoneb, connecting two embeddings. Use suitable loss instance
autoneb(x_data[5], x_data[17], fashion_UMAP_loss, config="configs/example_config.yaml")